In [1]:
import pandas as pd


In [2]:
pbp = pd.read_csv("../data_raw/play_by_play_2022.csv.gz")
part = pd.read_csv("../data_raw/pbp_participation_2022.csv")
rost = pd.read_csv("../data_raw/roster_weekly_2022.csv")

C:\Users\Owner\AppData\Local\Temp\ipykernel_31452\211303616.py:1: DtypeWarning: Columns (45,179,180,182,183,187,188,189,190,193,194,197,198,203,204,205,206,213,214,218,219,220,222,224,226,243,244,245,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp = pd.read_csv("../data_raw/play_by_play_2022.csv.gz")


In [3]:
pbp.shape, part.shape, rost.shape

((49434, 372), (50150, 20), (46163, 36))

In [4]:
pbp.columns
part.columns
rost.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'full_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'gsis_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week',
       'game_type', 'status_description_abbr', 'football_name', 'esb_id',
       'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club',
       'draft_number'],
      dtype='object')

In [5]:
pbp.columns.tolist()
part.columns.tolist()

['nflverse_game_id',
 'old_game_id',
 'play_id',
 'possession_team',
 'offense_formation',
 'offense_personnel',
 'defenders_in_box',
 'defense_personnel',
 'number_of_pass_rushers',
 'players_on_play',
 'offense_players',
 'defense_players',
 'n_offense',
 'n_defense',
 'ngs_air_yards',
 'time_to_throw',
 'was_pressure',
 'route',
 'defense_man_zone_type',
 'defense_coverage_type']

In [6]:
pbp.columns.tolist()

['play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'po

In [7]:
### Step 2: Explode participation to player–play rows
part_long = (
    part.assign(player_id=part["players_on_play"].str.split(";"))
        .explode("player_id")
        .loc[:, ["nflverse_game_id", "play_id", "player_id"]]
)

part_long.head()


,nflverse_game_id,play_id,player_id
0,2022_01_BUF_LA,1,NaN
1,2022_01_BUF_LA,41,52512
1,2022_01_BUF_LA,41,42818
1,2022_01_BUF_LA,41,48482
1,2022_01_BUF_LA,41,52835


In [8]:
part_long = (
    part_long
    .dropna(subset=["player_id"])
    .assign(player_id=lambda x: x["player_id"].astype(str))
)

In [9]:
part_long.head(), part_long["player_id"].isna().sum(), part_long["player_id"].dtype

(  nflverse_game_id  play_id player_id
 1   2022_01_BUF_LA       41     52512
 1   2022_01_BUF_LA       41     42818
 1   2022_01_BUF_LA       41     48482
 1   2022_01_BUF_LA       41     52835
 1   2022_01_BUF_LA       41     48772,
 np.int64(0),
 dtype('O'))

In [10]:
snaps = (
    part_long.groupby(["nflverse_game_id", "player_id"])
             .size()
             .reset_index(name="snaps")
)

snaps.head(), snaps.shape

(  nflverse_game_id player_id  snaps
 0  2022_01_BAL_NYJ     33099     84
 1  2022_01_BAL_NYJ     33131     49
 2  2022_01_BAL_NYJ     37145     54
 3  2022_01_BAL_NYJ     37778     32
 4  2022_01_BAL_NYJ     38557     60,
 (26381, 3))

In [11]:
st_plays = pbp.loc[
    pbp["play_type"].isin(["kickoff", "punt"]),
    ["game_id", "play_id"]
].assign(st_play=1)
st_plays.head(), st_plays.shape

(            game_id  play_id  st_play
 1   2022_01_BAL_NYJ       43        1
 6   2022_01_BAL_NYJ      172        1
 14  2022_01_BAL_NYJ      368        1
 21  2022_01_BAL_NYJ      533        1
 25  2022_01_BAL_NYJ      617        1,
 (5129, 3))

In [12]:
part_long = part_long.merge(
    st_plays,
    how="left",
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"]
)

part_long["st_play"] = part_long["st_play"].fillna(0).astype(int)

In [13]:
part_long["st_play"].value_counts()

st_play
0    895995
1    112802
Name: count, dtype: int64

In [14]:
st_snaps = (
    part_long.loc[part_long["st_play"] == 1]
    .groupby(["nflverse_game_id", "player_id"])
    .size()
    .reset_index(name="st_snaps")
)

st_snaps.head(), st_snaps.shape

(  nflverse_game_id player_id  st_snaps
 0  2022_01_BAL_NYJ     33131         1
 1  2022_01_BAL_NYJ     38701         2
 2  2022_01_BAL_NYJ     39470         5
 3  2022_01_BAL_NYJ     40095         6
 4  2022_01_BAL_NYJ     41243         2,
 (12875, 3))

In [15]:
player_game = snaps.merge(
    st_snaps,
    how="left",
    on=["nflverse_game_id", "player_id"]
)

player_game["st_snaps"] = player_game["st_snaps"].fillna(0).astype(int)

player_game.head(), player_game.shape

(  nflverse_game_id player_id  snaps  st_snaps
 0  2022_01_BAL_NYJ     33099     84         0
 1  2022_01_BAL_NYJ     33131     49         1
 2  2022_01_BAL_NYJ     37145     54         0
 3  2022_01_BAL_NYJ     37778     32         0
 4  2022_01_BAL_NYJ     38557     60         0,
 (26381, 4))

In [16]:
game_week = (
    pbp.loc[pbp["season"] == 2022, ["game_id", "season", "week"]]
      .drop_duplicates(subset=["game_id"])
      .reset_index(drop=True)
)

print(game_week.shape)
print(game_week["game_id"].nunique())
print(game_week["week"].min(), game_week["week"].max())

(284, 3)
284
1 22


In [17]:
player_game = player_game.rename(columns={"nflverse_game_id": "game_id"})

player_game = player_game.merge(
    game_week,
    on="game_id",
    how="left"
)

print(player_game.shape)
print(player_game["week"].isna().sum())
print(player_game[["game_id", "player_id", "snaps", "st_snaps", "season", "week"]].head())

(26381, 6)
0
           game_id player_id  snaps  st_snaps  season  week
0  2022_01_BAL_NYJ     33099     84         0    2022     1
1  2022_01_BAL_NYJ     33131     49         1    2022     1
2  2022_01_BAL_NYJ     37145     54         0    2022     1
3  2022_01_BAL_NYJ     37778     32         0    2022     1
4  2022_01_BAL_NYJ     38557     60         0    2022     1


In [18]:
print(rost.columns.tolist())

['season', 'team', 'position', 'depth_chart_position', 'jersey_number', 'status', 'full_name', 'first_name', 'last_name', 'birth_date', 'height', 'weight', 'college', 'gsis_id', 'espn_id', 'sportradar_id', 'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id', 'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week', 'game_type', 'status_description_abbr', 'football_name', 'esb_id', 'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'draft_number']


In [19]:
rost_meta = (
    rost.loc[rost["season"] == 2022, ["gsis_id", "season", "week", "full_name", "team", "position", "status"]]
      .rename(columns={"gsis_id": "player_id"})
      .drop_duplicates(subset=["player_id", "season", "week"])
      .reset_index(drop=True)
)

print(rost_meta.shape)
print(rost_meta[["player_id", "season", "week"]].duplicated().sum())
print(rost_meta[["full_name", "team", "position", "status"]].isna().mean())

(46154, 7)
0
full_name    0.0
team         0.0
position     0.0
status       0.0
dtype: float64


In [20]:
player_game = player_game.merge(
    rost_meta,
    on=["player_id", "season", "week"],
    how="left"
)

print(player_game.shape)
print(player_game[["full_name", "team", "position", "status"]].isna().sum())
print(player_game[["game_id", "player_id", "week", "team", "position", "snaps", "st_snaps"]].head())

(26381, 10)
full_name    26381
team         26381
position     26381
status       26381
dtype: int64
           game_id player_id  week team position  snaps  st_snaps
0  2022_01_BAL_NYJ     33099     1  NaN      NaN     84         0
1  2022_01_BAL_NYJ     33131     1  NaN      NaN     49         1
2  2022_01_BAL_NYJ     37145     1  NaN      NaN     54         0
3  2022_01_BAL_NYJ     37778     1  NaN      NaN     32         0
4  2022_01_BAL_NYJ     38557     1  NaN      NaN     60         0


In [21]:
# types
print("player_game dtypes:", player_game["player_id"].dtype, player_game["season"].dtype, player_game["week"].dtype)
print("rost_meta dtypes:  ", rost_meta["player_id"].dtype, rost_meta["season"].dtype, rost_meta["week"].dtype)

# quick samples
print("\nplayer_game player_id sample:", player_game["player_id"].head().tolist())
print("rost_meta  player_id sample:", rost_meta["player_id"].head().tolist())

# key overlap test (this is the decisive one)
pg_keys = set(zip(player_game["player_id"], player_game["season"], player_game["week"]))
rm_keys = set(zip(rost_meta["player_id"], rost_meta["season"], rost_meta["week"]))

print("\nkey overlap count:", len(pg_keys & rm_keys))

player_game dtypes: object int64 int64
rost_meta dtypes:   object int64 int64

player_game player_id sample: ['33099', '33131', '37145', '37778', '38557']
rost_meta  player_id sample: ['00-0019596', '00-0019596', '00-0019596', '00-0019596', '00-0019596']

key overlap count: 0


In [22]:
rost_meta = (
    rost.loc[rost["season"] == 2022, ["gsis_it_id", "season", "week", "full_name", "team", "position", "status"]]
      .rename(columns={"gsis_it_id": "player_id"})
      .drop_duplicates(subset=["player_id", "season", "week"])
      .reset_index(drop=True)
)

rost_meta["player_id"] = rost_meta["player_id"].astype(str)

print(rost_meta["player_id"].head().tolist())
print(rost_meta[["player_id", "season", "week"]].duplicated().sum())


['25511', '25511', '25511', '25511', '25511']
0


In [23]:
# remove the bad merge columns (all NaN)
player_game = player_game.drop(columns=["full_name", "team", "position", "status"], errors="ignore")

# merge roster metadata using the corrected ID system
player_game = player_game.merge(
    rost_meta,
    on=["player_id", "season", "week"],
    how="left"
)

print(player_game.shape)
print(player_game[["full_name", "team", "position", "status"]].isna().sum())
print(player_game[["game_id", "player_id", "week", "team", "position", "snaps", "st_snaps"]].head())

(26381, 10)
full_name    0
team         0
position     0
status       0
dtype: int64
           game_id player_id  week team position  snaps  st_snaps
0  2022_01_BAL_NYJ     33099     1  NYJ       QB     84         0
1  2022_01_BAL_NYJ     33131     1  BAL       DL     49         1
2  2022_01_BAL_NYJ     37145     1  BAL       LB     54         0
3  2022_01_BAL_NYJ     37778     1  BAL       LB     32         0
4  2022_01_BAL_NYJ     38557     1  BAL       OL     60         0


In [24]:
print(player_game[["full_name", "team", "position", "status"]].isna().sum())

full_name    0
team         0
position     0
status       0
dtype: int64


In [25]:
pbp_2022 = pbp.loc[pbp["season"] == 2022].copy()

pbp_2022["run_play"] = (pbp_2022["play_type"] == "run").astype(int)

print(pbp_2022["run_play"].value_counts())

run_play
0    34397
1    15037
Name: count, dtype: int64


In [26]:
part_long = part_long.merge(
    pbp_2022[["game_id", "play_id", "run_play"]],
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"],
    how="left"
)

print(part_long["run_play"].isna().sum())
print(part_long["run_play"].value_counts())

0
run_play
0    677998
1    330799
Name: count, dtype: int64


In [27]:
run_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["run_play"]
      .sum()
      .reset_index(name="run_snaps")
)

print(run_snaps.shape)
print(run_snaps["run_snaps"].describe())

(26381, 3)
count    26381.000000
mean        12.539290
std         11.144962
min          0.000000
25%          1.000000
50%         11.000000
75%         21.000000
max         48.000000
Name: run_snaps, dtype: float64


In [28]:
run_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["run_play"]
      .sum()
      .reset_index(name="run_snaps")
)

print(run_snaps.shape)
print(run_snaps["run_snaps"].describe())

(26381, 3)
count    26381.000000
mean        12.539290
std         11.144962
min          0.000000
25%          1.000000
50%         11.000000
75%         21.000000
max         48.000000
Name: run_snaps, dtype: float64


In [29]:
player_game = player_game.merge(
    run_snaps,
    left_on=["game_id", "player_id"],
    right_on=["nflverse_game_id", "player_id"],
    how="left"
)

# clean up the extra key column
player_game = player_game.drop(columns=["nflverse_game_id"])

print(player_game.shape)
print(player_game["run_snaps"].isna().sum())
print(player_game[["snaps", "run_snaps"]].head())

(26381, 11)
0
   snaps  run_snaps
0     84         17
1     49         12
2     54          8
3     32         13
4     60         19


In [30]:
pbp_2022["goal_to_go"] = pbp_2022["goal_to_go"].fillna(0).astype(int)

print(pbp_2022["goal_to_go"].value_counts())

goal_to_go
0    47013
1     2421
Name: count, dtype: int64


In [31]:
part_long = part_long.merge(
    pbp_2022[["game_id", "play_id", "goal_to_go"]],
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"],
    how="left"
)

print(part_long["goal_to_go"].isna().sum())
print(part_long["goal_to_go"].value_counts())

0
goal_to_go
0    955533
1     53264
Name: count, dtype: int64


In [32]:
goal_to_go_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["goal_to_go"]
      .sum()
      .reset_index(name="goal_to_go_snaps")
)

print(goal_to_go_snaps.shape)
print(goal_to_go_snaps["goal_to_go_snaps"].describe())

(26381, 3)
count    26381.000000
mean         2.019029
std          2.702794
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max         20.000000
Name: goal_to_go_snaps, dtype: float64


In [33]:
print(goal_to_go_snaps.shape)
print(goal_to_go_snaps["goal_to_go_snaps"].describe())

(26381, 3)
count    26381.000000
mean         2.019029
std          2.702794
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max         20.000000
Name: goal_to_go_snaps, dtype: float64


In [34]:
player_game = (
    player_game
      .merge(
          goal_to_go_snaps,
          left_on=["game_id", "player_id"],
          right_on=["nflverse_game_id", "player_id"],
          how="left"
      )
      .drop(columns=["nflverse_game_id"])
)

print(player_game.shape)
print(player_game["goal_to_go_snaps"].isna().sum())
print(player_game[["snaps", "goal_to_go_snaps"]].head())

(26381, 12)
0
   snaps  goal_to_go_snaps
0     84                 7
1     49                 1
2     54                 1
3     32                 3
4     60                 0


In [35]:
pbp_2022["short_yardage"] = (pbp_2022["ydstogo"] <= 2).astype(int)

print(pbp_2022["short_yardage"].value_counts())

short_yardage
0    37509
1    11925
Name: count, dtype: int64


In [36]:
# keep only the canonical game id column name for part_long
cols_to_drop = [c for c in part_long.columns if c.startswith("game_id")]
print("Dropping:", cols_to_drop)

part_long = part_long.drop(columns=cols_to_drop, errors="ignore")

Dropping: ['game_id_x', 'game_id_y', 'game_id']


In [37]:
goal_to_go_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["goal_to_go"]
      .sum()
      .reset_index(name="goal_to_go_snaps")
)

print(goal_to_go_snaps.shape)
print(goal_to_go_snaps["goal_to_go_snaps"].describe())

(26381, 3)
count    26381.000000
mean         2.019029
std          2.702794
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max         20.000000
Name: goal_to_go_snaps, dtype: float64


In [38]:
# If goal_to_go_snaps already exists, don't merge again
if "goal_to_go_snaps" not in player_game.columns:
    player_game = (
        player_game
          .merge(
              goal_to_go_snaps,
              left_on=["game_id", "player_id"],
              right_on=["nflverse_game_id", "player_id"],
              how="left",
              validate="one_to_one" 
          )
          .drop(columns=["nflverse_game_id"])
    )

print(player_game["goal_to_go_snaps"].isna().sum())
print(player_game[["snaps", "goal_to_go_snaps"]].head())

0
   snaps  goal_to_go_snaps
0     84                 7
1     49                 1
2     54                 1
3     32                 3
4     60                 0


In [39]:
part_long = part_long.merge(
    pbp_2022[["game_id", "play_id", "short_yardage"]],
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"],
    how="left",
    validate="many_to_one"
)

# drop the right-side join key so we don't have duplicates again
part_long = part_long.drop(columns=["game_id"])

print(part_long["short_yardage"].isna().sum())
print(part_long["short_yardage"].value_counts())

0
short_yardage
0    825110
1    183687
Name: count, dtype: int64


In [40]:
short_yardage_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["short_yardage"]
      .sum()
      .reset_index(name="short_yardage_snaps")
)

print(short_yardage_snaps.shape)
print(short_yardage_snaps["short_yardage_snaps"].describe())

(26381, 3)
count    26381.000000
mean         6.962852
std          4.502950
min          0.000000
25%          3.000000
50%          6.000000
75%         10.000000
max         32.000000
Name: short_yardage_snaps, dtype: float64


In [41]:
player_game = (
    player_game
      .merge(
          short_yardage_snaps,
          left_on=["game_id", "player_id"],
          right_on=["nflverse_game_id", "player_id"],
          how="left"
      )
      .drop(columns=["nflverse_game_id"])
)

print(player_game.shape)
print(player_game["short_yardage_snaps"].isna().sum())
print(player_game[["snaps", "short_yardage_snaps"]].head())

(26381, 13)
0
   snaps  short_yardage_snaps
0     84                    4
1     49                    6
2     54                    2
3     32                    1
4     60                    7


In [42]:
pbp_2022["red_zone"] = (pbp_2022["yardline_100"] <= 20).astype(int)

print(pbp_2022["red_zone"].value_counts())

red_zone
0    41828
1     7606
Name: count, dtype: int64


In [43]:
part_long = part_long.merge(
    pbp_2022[["game_id", "play_id", "red_zone"]],
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"],
    how="left",
    validate="many_to_one"
)

# drop the right-side key 
part_long = part_long.drop(columns=["game_id"])

print(part_long["red_zone"].isna().sum())
print(part_long["red_zone"].value_counts())

0
red_zone
0    841505
1    167292
Name: count, dtype: int64


In [44]:
red_zone_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["red_zone"]
      .sum()
      .reset_index(name="red_zone_snaps")
)

print(red_zone_snaps.shape)
print(red_zone_snaps["red_zone_snaps"].describe())

(26381, 3)
count    26381.000000
mean         6.341382
std          5.611566
min          0.000000
25%          2.000000
50%          5.000000
75%         10.000000
max         36.000000
Name: red_zone_snaps, dtype: float64


In [45]:
player_game = (
    player_game
      .merge(
          red_zone_snaps,
          left_on=["game_id", "player_id"],
          right_on=["nflverse_game_id", "player_id"],
          how="left"
      )
      .drop(columns=["nflverse_game_id"])
)

print(player_game.shape)
print(player_game["red_zone_snaps"].isna().sum())
print(player_game[["snaps", "red_zone_snaps", "goal_to_go_snaps"]].head())

(26381, 14)
0
   snaps  red_zone_snaps  goal_to_go_snaps
0     84              13                 7
1     49               8                 1
2     54               7                 1
3     32               3                 3
4     60              10                 0


In [46]:
contact_terms = [
    "Unnecessary Roughness",
    "Roughing the Passer",
    "Roughing the Kicker",
    "Targeting",
    "Blindside Block"
]

pbp_2022["penalty_contact"] = (
    pbp_2022["penalty_type"]
      .fillna("")
      .str.contains("|".join(contact_terms), case=False)
      .astype(int)
)

print(pbp_2022["penalty_contact"].value_counts())

penalty_contact
0    49156
1      278
Name: count, dtype: int64


In [47]:
part_long = part_long.merge(
    pbp_2022[["game_id", "play_id", "penalty_contact"]],
    left_on=["nflverse_game_id", "play_id"],
    right_on=["game_id", "play_id"],
    how="left",
    validate="many_to_one"
)

# drop the right-side key 
part_long = part_long.drop(columns=["game_id"])

print(part_long["penalty_contact"].isna().sum())
print(part_long["penalty_contact"].value_counts())

0
penalty_contact
0    1002684
1       6113
Name: count, dtype: int64


In [48]:
penalty_contact_snaps = (
    part_long
      .groupby(["nflverse_game_id", "player_id"])["penalty_contact"]
      .sum()
      .reset_index(name="penalty_contact_snaps")
)

print(penalty_contact_snaps.shape)
print(penalty_contact_snaps["penalty_contact_snaps"].describe())

(26381, 3)
count    26381.000000
mean         0.231720
std          0.503532
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: penalty_contact_snaps, dtype: float64


In [49]:
player_game = (
    player_game
      .merge(
          penalty_contact_snaps,
          left_on=["game_id", "player_id"],
          right_on=["nflverse_game_id", "player_id"],
          how="left"
      )
      .drop(columns=["nflverse_game_id"])
)

print(player_game.shape)
print(player_game["penalty_contact_snaps"].isna().sum())
print(
    player_game[
        ["snaps", "run_snaps", "short_yardage_snaps",
         "goal_to_go_snaps", "red_zone_snaps",
         "penalty_contact_snaps"]
    ].head()
)

(26381, 15)
0
   snaps  run_snaps  short_yardage_snaps  goal_to_go_snaps  red_zone_snaps  \
0     84         17                    4                 7              13   
1     49         12                    6                 1               8   
2     54          8                    2                 1               7   
3     32         13                    1                 3               3   
4     60         19                    7                 0              10   

   penalty_contact_snaps  
0                      0  
1                      0  
2                      0  
3                      0  
4                      0  


In [50]:
from pathlib import Path

# create the root data_processed folder if needed
Path("../data_processed").mkdir(parents=True, exist_ok=True)

# save to the root data_processed folder
player_game.to_parquet(
    "../data_processed/player_game_2022.parquet",
    index=False
)